Look at the data


In [1]:
import numpy as np
import pandas as pd
import torch
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data.csv')
df.head()


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables


In [3]:
# Check for null values in each column
null_counts = df.isnull().sum()
print("\nColumns with null values:")
print(null_counts[null_counts > 0])



Columns with null values:
days_since_prior_order    124342
dtype: int64


In [4]:
# Replace null values with 0
df = df.fillna(0)

# Verify the replacement
null_counts_after = df.isnull().sum()
print("\nColumns with null values after replacement:")
print(null_counts_after[null_counts_after > 0])



Columns with null values after replacement:
Series([], dtype: int64)


In [5]:
df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,0.0,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,0.0,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,0.0,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,0.0,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,0.0,83,5,0,4,produce,fresh vegetables


In [6]:
# Remove product_id and user_id columns
df = df.drop(['add_to_cart_order', 'department_id'], axis=1)

# Display the first few rows to verify the columns were removed
df.head()


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered,department,product_name
0,2425083,49125,1,2,18,0.0,17,0,pantry,baking ingredients
1,2425083,49125,1,2,18,0.0,91,0,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,0.0,36,0,dairy eggs,butter
3,2425083,49125,1,2,18,0.0,83,0,produce,fresh vegetables
4,2425083,49125,1,2,18,0.0,83,0,produce,fresh vegetables


In [7]:
# Create a dictionary to map numbers to weekdays (0 = Sunday, 1 = Monday, etc.)
weekday_map = {
    0: 'Sunday',
    1: 'Monday', 
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday'
}

# Map the order_dow numbers to weekday names
df['weekday'] = df['order_dow'].map(weekday_map)

# Display the first few rows to verify the mapping
df.head()


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered,department,product_name,weekday
0,2425083,49125,1,2,18,0.0,17,0,pantry,baking ingredients,Tuesday
1,2425083,49125,1,2,18,0.0,91,0,dairy eggs,soy lactosefree,Tuesday
2,2425083,49125,1,2,18,0.0,36,0,dairy eggs,butter,Tuesday
3,2425083,49125,1,2,18,0.0,83,0,produce,fresh vegetables,Tuesday
4,2425083,49125,1,2,18,0.0,83,0,produce,fresh vegetables,Tuesday


In [8]:
df = df.drop(['order_dow'], axis=1)

df.head()


,order_id,user_id,order_number,order_hour_of_day,days_since_prior_order,product_id,reordered,department,product_name,weekday
0,2425083,49125,1,18,0.0,17,0,pantry,baking ingredients,Tuesday
1,2425083,49125,1,18,0.0,91,0,dairy eggs,soy lactosefree,Tuesday
2,2425083,49125,1,18,0.0,36,0,dairy eggs,butter,Tuesday
3,2425083,49125,1,18,0.0,83,0,produce,fresh vegetables,Tuesday
4,2425083,49125,1,18,0.0,83,0,produce,fresh vegetables,Tuesday


In [9]:
# Create a dictionary to map binary values to 'yes'/'no'
reorder_map = {
    0: 'no',
    1: 'yes'
}

# Map the reordered column values
df['reordered'] = df['reordered'].map(reorder_map)

# Display the first few rows to verify the mapping
df.head()


,order_id,user_id,order_number,order_hour_of_day,days_since_prior_order,product_id,reordered,department,product_name,weekday
0,2425083,49125,1,18,0.0,17,no,pantry,baking ingredients,Tuesday
1,2425083,49125,1,18,0.0,91,no,dairy eggs,soy lactosefree,Tuesday
2,2425083,49125,1,18,0.0,36,no,dairy eggs,butter,Tuesday
3,2425083,49125,1,18,0.0,83,no,produce,fresh vegetables,Tuesday
4,2425083,49125,1,18,0.0,83,no,produce,fresh vegetables,Tuesday


In [10]:
# Create a pivot table to count orders by weekday
weekday_counts = df.groupby('weekday').size().reset_index(name='count')

# Sort the weekdays in chronological order
weekday_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
weekday_counts['weekday'] = pd.Categorical(weekday_counts['weekday'], categories=weekday_order, ordered=True)
weekday_counts = weekday_counts.sort_values('weekday')

# Display the weekday counts
weekday_counts


# Save the weekday counts to an Excel file
weekday_counts.to_excel('weekday_order_counts.xlsx', index=False)




In [12]:
# Create a combination of weekday and hour
hour_weekday_counts = df.groupby(['weekday', 'order_hour_of_day']).size().reset_index(name='count')

# Sort by count in descending order and get top 10
top_10_combinations = hour_weekday_counts.sort_values('count', ascending=False).head(10)

# Display the results
print("Top 10 most common weekday and hour combinations:")
print(top_10_combinations)





Top 10 most common weekday and hour combinations:
   weekday  order_hour_of_day  count
85  Sunday                 13  36586
86  Sunday                 14  35256
87  Sunday                 15  34683
83  Sunday                 11  34523
34  Monday                 10  34326
84  Sunday                 12  33670
82  Sunday                 10  32921
88  Sunday                 16  31776
33  Monday                  9  31299
35  Monday                 11  30120


In [13]:
# Sort by count in ascending order and get bottom 10
bottom_10_combinations = hour_weekday_counts.sort_values('count', ascending=True).head(10)

# Display the results
print("\nBottom 10 least common weekday and hour combinations:")
print(bottom_10_combinations)



Bottom 10 least common weekday and hour combinations:
       weekday  order_hour_of_day  count
147  Wednesday                  3    254
148  Wednesday                  4    342
75      Sunday                  3    415
3       Friday                  3    425
122    Tuesday                  2    430
76      Sunday                  4    438
99    Thursday                  3    441
123    Tuesday                  3    459
124    Tuesday                  4    463
51    Saturday                  3    490


In [11]:
# Create separate DataFrames for each day of the week
sunday_orders = df[df['weekday'] == 'Sunday']
monday_orders = df[df['weekday'] == 'Monday'] 
tuesday_orders = df[df['weekday'] == 'Tuesday']
wednesday_orders = df[df['weekday'] == 'Wednesday']
thursday_orders = df[df['weekday'] == 'Thursday']
friday_orders = df[df['weekday'] == 'Friday']
saturday_orders = df[df['weekday'] == 'Saturday']

# Save each day's data to separate Excel files
sunday_orders.to_excel('sunday_orders.xlsx', index=False)
monday_orders.to_excel('monday_orders.xlsx', index=False)
tuesday_orders.to_excel('tuesday_orders.xlsx', index=False)
wednesday_orders.to_excel('wednesday_orders.xlsx', index=False)
thursday_orders.to_excel('thursday_orders.xlsx', index=False)
friday_orders.to_excel('friday_orders.xlsx', index=False)
saturday_orders.to_excel('saturday_orders.xlsx', index=False)

# Display the number of orders for each day
print("Number of orders by day:")
print(f"Sunday: {len(sunday_orders)}")
print(f"Monday: {len(monday_orders)}")
print(f"Tuesday: {len(tuesday_orders)}")
print(f"Wednesday: {len(wednesday_orders)}")
print(f"Thursday: {len(thursday_orders)}")
print(f"Friday: {len(friday_orders)}")
print(f"Saturday: {len(saturday_orders)}")


Number of orders by day:
Sunday: 391831
Monday: 349236
Tuesday: 261912
Wednesday: 238730
Thursday: 234884
Friday: 262157
Saturday: 280751


In [16]:

# Calculate the average days since last order
average_days = df['days_since_prior_order'].mean()

print(f"\nAverage days between orders: {average_days:.2f}")

# Calculate the mode (most common value) for days between orders
mode_days = df['days_since_prior_order'].mode()[0]

print(f"Most common number of days between orders: {mode_days:.2f}")




Average days between orders: 10.68
Most common number of days between orders: 7.00
